In [9]:
import pandas as pd
import numpy as np

data = pd.read_csv('..//data//created//block_neigh_2.csv')
blocks = pd.read_csv('..//data//census//tl_2010_06075_tabblock10.csv')
ped_vol = pd.read_csv('..//data//sf_open//ped_vol_blockid.csv')
#land_use = pd.read_csv('data_sf_open//land_use_blockid.csv')
income = pd.read_csv('..//data//census//income//ACS_16_5YR_B19013_with_ann.csv')
pop = pd.read_csv('..//data//census//total_population//ACS_16_5YR_B01003_with_ann.csv')
emp = pd.read_csv('..//data//census//employment//ACS_16_5YR_B23027_with_ann.csv')
poverty = pd.read_csv('..//data//census//poverty//ACS_16_5YR_B99172_with_ann.csv')
age_sex = pd.read_csv('..//data//census//age_by_sex//ACS_16_5YR_B01002_with_ann.csv')
encamp = pd.read_csv('..//data//created//encamp_feature.csv')

In [10]:
blocks['bg_num'] = blocks['GEOID10'].apply(lambda x: str(x)[:-4])
blocks['bg_b_num'] = blocks['BLOCKCE10'].apply(lambda x: str(x)[0])
blocks['bg_num'] = (blocks['bg_num'] + blocks['bg_b_num']).astype(int)

blocks_id = blocks.loc[:,[
    
    'GEOID10',
    'BLOCKCE10',
    'bg_num'
]]

blocks_id.columns = ['block_fips', 'block_num', 'bg_num']

In [11]:
sq_ft_use = pd.pivot_table(land_use,
                           index='block_fips',
                           columns='LANDUSE',
                           values = 'BLDGSQFT').reset_index()

sq_ft_use = sq_ft_use.fillna(0)

sq_ft_use = sq_ft_use.groupby('block_fips').mean().reset_index()

sq_ft_use = sq_ft_use.loc[:,[
    
    'block_fips',
    'RESIDENT',
    'MIXRES',
    'RETAIL/ENT',
    'CIE',
    'MED',
    'MIPS',
    'PDR',
    'VISITOR',
    'VACANT',
    'ROW',
    'OPENSPACE'
]]

sq_ft_use.columns = [
    
    'block_fips', 
    'res_sqft',
    'mixres_sqft',
    'retail_sqft',
    'cie_sqft',
    'medical_sqft',
    'office_sqft',
    'indus_sqft',
    'hotel_sqft',
    'vacant_sqft',
    'row_sqft',
    'openspace_sqft'
]

NameError: name 'land_use' is not defined

In [12]:
income = pd.read_csv('..//data//census//income//ACS_16_5YR_B19013_with_ann.csv')
pop = pd.read_csv('..//data//census//total_population//ACS_16_5YR_B01003_with_ann.csv')
emp = pd.read_csv('..//data//census//employment//ACS_16_5YR_B23027_with_ann.csv')
poverty = pd.read_csv('..//data//census//poverty//ACS_16_5YR_B99172_with_ann.csv')
age_sex = pd.read_csv('..//data//census//age_by_sex//ACS_16_5YR_B01002_with_ann.csv')
race = pd.read_csv('..//data//census//race//ACS_16_5YR_B02001_with_ann.csv')

# income
income = income[income['GEO.id']!='Id']
income['GEO.id2'] = income['GEO.id2'].astype(int)
income = income.merge(blocks_id, left_on = 'GEO.id2', right_on = 'bg_num')

income = income.loc[:,[
    
    'block_fips',
    'HD01_VD01',
]]

income.columns = ['block_fips', 'HH_income']

# population
pop = pop[pop['GEO.id']!='Id']
pop['GEO.id2'] = pop['GEO.id2'].astype(int)
pop = pop.merge(blocks_id, left_on = 'GEO.id2', right_on = 'bg_num')

pop = pop.loc[:,[
    
    'block_fips',
    'HD01_VD01',
]]

pop.columns = ['block_fips', 'total_pop']

# employment
emp = emp[emp['GEO.id']!='Id']
emp['GEO.id2'] = emp['GEO.id2'].astype(int)
emp = emp.merge(blocks_id, left_on = 'GEO.id2', right_on = 'bg_num')

emp = emp.loc[:,[
    
    'block_fips',
    'HD01_VD01',
    'HD01_VD11',
    'HD01_VD16',
    'HD01_VD21',
    'HD01_VD26'
]]

emp.columns = ['block_fips', 'total', 'unemp_20_24', 'unemp_25_44', 'unemp_45_54', 'unemp_55_64']

emp['unemp_20_24'] = emp['unemp_20_24'].astype(int) / emp['total'].astype(int)
emp['unemp_25_44'] = emp['unemp_25_44'].astype(int) / emp['total'].astype(int)
emp['unemp_45_54'] = emp['unemp_45_54'].astype(int) / emp['total'].astype(int)
emp['unemp_55_64'] = emp['unemp_55_64'].astype(int) / emp['total'].astype(int)

emp.drop('total', axis=1, inplace=True)

# age by sex
age_sex = age_sex[age_sex['GEO.id']!='Id']
age_sex['GEO.id2'] = age_sex['GEO.id2'].astype(int)
age_sex = age_sex.merge(blocks_id, left_on = 'GEO.id2', right_on = 'bg_num')

age_sex = age_sex.loc[:,[
    
    'block_fips',
    'HD01_VD02',
    'HD01_VD03',
    'HD01_VD04'
]]

age_sex.columns = ['block_fips', 'total_age', 'male_age', 'fem_age']

# race
race = race[race['GEO.id']!='Id']
race['GEO.id2'] = race['GEO.id2'].astype(int)
race = race.merge(blocks_id, left_on = 'GEO.id2', right_on = 'bg_num')

race = race.loc[:,[
    
    'block_fips',
    'HD01_VD01',
    'HD01_VD02',
    'HD01_VD03',
    'HD01_VD05',
    'HD01_VD07'
]]

race.columns = ['block_fips', 'total', 'white', 'black', 'asian', 'other']

race['white'] = race['white'].astype(int) / race['total'].astype(int)
race['black'] = race['black'].astype(int) / race['total'].astype(int)
race['asian'] = race['asian'].astype(int) / race['total'].astype(int)
race['other'] = race['other'].astype(int) / race['total'].astype(int)

race.drop('total', axis=1, inplace=True)

In [13]:
ped_vol = ped_vol.loc[:,[
    
    'block_fips',
    'TOTEMP2',
    'UNIVPROX',
    'SIGNALIZED', 
    'PKGMETERS', 
    'MAXPCTSLPE',
    'MODEL6_VOL', 
    'HH_PEDMODE', 
    'PCOL_04_09', 
    'PCOL_RATE'
]]

ped_vol = ped_vol.groupby('block_fips').mean().reset_index()

In [14]:
block_meta = blocks_id.merge(ped_vol, on = 'block_fips', how='left')
#block_meta = block_meta.merge(sq_ft_use, on = 'block_fips')
block_meta = block_meta.merge(income, on = 'block_fips')
block_meta = block_meta.merge(pop, on = 'block_fips')
block_meta = block_meta.merge(emp, on = 'block_fips')
block_meta = block_meta.merge(age_sex, on = 'block_fips')
block_meta = block_meta.merge(race, on = 'block_fips')

block_meta = block_meta.drop(['block_num', 'bg_num'], axis=1)

In [15]:
block_meta

,block_fips,TOTEMP2,UNIVPROX,SIGNALIZED,PKGMETERS,MAXPCTSLPE,MODEL6_VOL,HH_PEDMODE,PCOL_04_09,PCOL_RATE,...,unemp_25_44,unemp_45_54,unemp_55_64,total_age,male_age,fem_age,white,black,asian,other
0,60750213002002,227.000000,0.0,0.000000,0.000000,8.000000,4.650100e+04,2024.000000,0.000000,0.000000,...,0.044681,0.004965,0.037589,40.6,41.2,40.1,0.804386,0.000000,0.107884,0.016005
1,60750213002000,2117.000000,0.0,0.000000,1.000000,7.500000,1.564455e+05,2079.500000,0.000000,0.000000,...,0.044681,0.004965,0.037589,40.6,41.2,40.1,0.804386,0.000000,0.107884,0.016005
2,60750205003004,1980.000000,0.0,0.000000,1.000000,8.000000,4.838700e+04,2223.000000,0.000000,0.000000,...,0.016164,0.035560,0.037716,44.5,49.8,30.6,0.846686,0.039565,0.081108,0.000000
3,60750212001003,1989.500000,0.0,0.000000,1.000000,6.500000,1.497195e+05,2201.500000,0.000000,0.000000,...,0.000000,0.013369,0.068182,45.4,47.4,44.2,0.768765,0.000000,0.154964,0.031477
4,60750211004001,2762.000000,0.0,0.000000,0.000000,4.000000,2.167030e+05,2213.000000,0.000000,0.000000,...,0.014269,0.046373,0.040428,44.5,45.0,43.0,0.748713,0.000000,0.147271,0.041195
5,60750211004002,2631.000000,0.0,0.000000,0.000000,3.000000,1.649210e+05,2246.000000,0.000000,0.000000,...,0.014269,0.046373,0.040428,44.5,45.0,43.0,0.748713,0.000000,0.147271,0.041195
6,60750216001005,386.666667,0.0,0.000000,0.000000,16.333333,4.463800e+04,2119.333333,0.000000,0.000000,...,0.019033,0.000000,0.035494,40.9,32.6,46.6,0.762136,0.028244,0.123566,0.006620
7,60750217002017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.013423,0.000000,44.8,48.9,44.2,0.767956,0.046961,0.105893,0.046041
8,60750216001012,384.500000,0.0,0.000000,0.000000,11.500000,6.906800e+04,1633.000000,0.000000,0.000000,...,0.019033,0.000000,0.035494,40.9,32.6,46.6,0.762136,0.028244,0.123566,0.006620
9,60750217003010,586.666667,0.0,0.000000,0.333333,11.666667,1.392413e+05,1232.333333,0.000000,0.000000,...,0.055944,0.034329,0.047044,41.6,37.9,43.9,0.551136,0.153409,0.126550,0.126550


In [16]:
block_meta.to_csv('..//data//created//block_meta.csv', index = False)